In [1]:
import pandas as pd
import numpy as np

# Random dataset

In [183]:
def random_bool(size):
    return np.random.rand(size) > 0.5

def random_int(min, max, size):
    return np.random.randint(min, max, size=size)

def random_float(min, max, size):
    return np.random.uniform(min, max, size=size)


def index_id(max):
    list = []
    for i in range(1, max):
        list.append('ID' + str(i))
    return list

In [184]:
pre_op = pd.DataFrame({'IDno': index_id(501), 'Død inden for 1 år af operation': random_bool(500), 'age': random_int(18, 89, 500), 'sex': random_bool(500), 'Respiratory Disease': random_bool(500),
       'Circulatory Organ Disease': random_bool(500), 'Type 1 Diabetes': random_bool(500), 'Type 2 Diabetes': random_bool(500),
       'Other metabolic diseases': random_bool(500), 'Other operation': random_bool(500), 'Genital or urine related diseases': random_bool(500), 'Vægt': random_float(35, 190,500), 'KFN': random_bool(500), 'KFX': random_bool(500), 'KFK': random_bool(500), 'KFM': random_bool(500),
       'KFF': random_bool(500), 'KFP': random_bool(500), 'KFC': random_bool(500), 'KFW': random_bool(500), 'KFJ': random_bool(500), 'Hæmoglobin': random_float(4,12,500), 'Leukocytter': random_float(1, 125, 500),
       'Trombocytter': random_float(20, 1000, 500)})
pre_op = pre_op.replace(True, 1).replace(False, 0)

# Predicting Mortality with pre_op data

## LR

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation'], pre_op['Død inden for 1 år af operation'], test_size=0.3, random_state=42)

def lr_pred(X_train, y_train, X_test):
    clf = LogisticRegression().fit(X_train, y_train)
    return clf.predict_proba(X_test)

## Random forrest

## Gradient boost


## MLP

# Grouped cross-validation

In [212]:
from sklearn.model_selection import GroupKFold

X = pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation']
y = pre_op['Død inden for 1 år af operation']

outer_cv = GroupKFold(n_splits = 5)
inner_cv = GroupKFold(n_splits = 5)

for outer_train_idx, outer_test_idx in outer_cv.split(X, y, groups = pre_op['IDno']):
    X_outer_train, y_outer_train = X.loc[outer_train_idx], y.loc[outer_train_idx]
    X_outer_test, y_outer_test = X.loc[outer_test_idx], y.loc[outer_test_idx]

    for inner_train_idx, inner_test_idx in inner_cv.split(X_outer_train, y_outer_train, groups=pre_op['IDno']):
        X_inner_train, y_inner_train = X_outer_train.loc[inner_train_idx], y_outer_train.loc[inner_train_idx]
        X_inner_test, y_inner_test = X_outer_train.loc[inner_test_idx], y_outer_train.loc[inner_test_idx]

        print('works')

ValueError: Found input variables with inconsistent numbers of samples: [400, 400, 500]